In [ ]:
#Se importan librerías iniciales
!pip install rlxutils
import os
import pandas as pd
import numpy as np
from rlxutils import subplots
import matplotlib.pyplot as plt

#Se importan otras librerías requeridas para pruebas
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns
import unicodedata


#Se cargan los datos de Kagle
os.environ['KAGGLE_CONFIG_DIR'] = '.'
!chmod 600 ./kaggle.json
!kaggle competitions download -c udea-ai4eng-20242

#Se extraen y descomprimen archivos
!unzip udea*.zip > /dev/null

#Se carga el archivo "train.csv"
GetFileTrain = pd.read_csv("train.csv")

#IMPORTANTE!!
#Se excluye la columna "ID" en el análisis
#DataSet = Data
Data = GetFileTrain.drop(columns=['ID'])

#COLUMNA PERIODO
#Modificar la columna 'PERIODO' para que solo contenga el valor del año
Data['PERIODO'] = Data['PERIODO'].astype(str).str[:4].astype(int)

#Diccionario de mapeo para los valores de PERIODO
desempeno_periodo_map = {
    2018: 1,
    2019: 2,
    2020: 3,
    2021: 4
}
#Crear la columna DESEMPEÑO_PERIODO basada en los valores de PERIODO
Data['DESEMPEÑO_PERIODO'] = Data['PERIODO'].map(desempeno_periodo_map)
#Verificar primeras filas
print(Data[['PERIODO', 'DESEMPEÑO_PERIODO']].head(10))
#Crear columnas binarias para cada año en la columna 'PERIODO' con prefijo 'PERIODO'
Data = pd.get_dummies(Data, columns=['PERIODO'], prefix='PERIODO')
#Filtrar las columnas que empiezan con 'PERIODO_'
periodo_columns = [col for col in Data.columns if col.startswith('PERIODO_')]
#Convertir valores True/False a 1 y 0 solo en las columnas filtradas
Data[periodo_columns] = Data[periodo_columns].astype(int)
#Verificar los cambios
for col in periodo_columns:
    print(f"{col}:\n", Data[col].head(10))
    print("\n" + "-"*40 + "\n")
print("--------------------------------------------------------")


#COLUMNA ESTU_PRGM_ACADEMICO
#Función para eliminar tildes y normalizar el texto
def normalizar_texto(texto):
    if isinstance(texto, str):
        texto = texto.upper()
        texto = ''.join((c for c in unicodedata.normalize('NFD', texto) if unicodedata.category(c) != 'Mn'))
    return texto
#Cargar el archivo Excel "Programas.xlsx" y seleccionar la hoja "Sheet1"
programas_df = pd.read_excel('Programas.xlsx', sheet_name='Sheet1')
#Asegurarse de que los datos en la columna "ESTU_PRGM_ACADEMICO" estén normalizados
Data['ESTU_PRGM_ACADEMICO'] = Data['ESTU_PRGM_ACADEMICO'].apply(normalizar_texto)
#Normalizar también la columna 'Programa' y 'Campo de Conocimiento' en el archivo Excel
programas_df['Programa'] = programas_df['Programa'].apply(normalizar_texto)
programas_df['Campo de Conocimiento'] = programas_df['Campo de Conocimiento'].apply(normalizar_texto)
#Crear un diccionario para coincidencias exactas entre 'Programa' y 'Campo de Conocimiento'
reemplazos = dict(zip(programas_df['Programa'], programas_df['Campo de Conocimiento']))
#Reemplazar las coincidencias exactas en la columna 'ESTU_PRGM_ACADEMICO'
Data['ESTU_PRGM_ACADEMICO'] = Data['ESTU_PRGM_ACADEMICO'].replace(reemplazos)
#Verificar los primeros valores después del reemplazo
print(Data['ESTU_PRGM_ACADEMICO'].head(10))
#Diccionario de mapeo para 'DESEMPEÑO_PRGM_ACADEMICO'
desempeno_prgm_map = {
    'SALUD': 1,
    'CIENCIAS SOCIALES Y HUMANIDADES': 2,
    'CIENCIAS ECONOMICAS Y ADMINISTRATIVAS': 3,
    'AGRONOMIA, VETERINARIA Y AFINES': 4,
    'CIENCIAS NATURALES E INGENIERIA': 5,
    'EDUCACION Y PEDAGOGIA': 6,
    'ARTE, DISENO, CULTURA Y DEPORTE': 7
}
#Crear la nueva columna 'DESEMPEÑO_PRGM_ACADEMICO' basada en 'ESTU_PRGM_ACADEMICO'
Data['DESEMPEÑO_PRGM_ACADEMICO'] = Data['ESTU_PRGM_ACADEMICO'].map(desempeno_prgm_map)
#Verificar las primeras filas
print(Data[['ESTU_PRGM_ACADEMICO', 'DESEMPEÑO_PRGM_ACADEMICO']].head(10))
#Crear columnas binarias para cada programa en 'ESTU_PRGM_ACADEMICO' con prefijo 'PRGM_'
Data = pd.get_dummies(Data, columns=['ESTU_PRGM_ACADEMICO'], prefix='PRGM')
#Filtrar las columnas que empiezan con 'PRGM_'
program_columns = [col for col in Data.columns if col.startswith('PRGM_')]
#Convertir valores True/False a 1 y 0 solo en las columnas filtradas
Data[program_columns] = Data[program_columns].astype(int)
#Verificar los cambios en las primeras 10 filas de cada nueva columna
for col in program_columns:
    print(f"{col}:\n", Data[col].head(10))
    print("\n" + "-"*40 + "\n")
print("--------------------------------------------------------")

# OLUMNA ESTU_VALORMATRICULAUNIVERSIDAD
print("COLUMNA ESTU_VALORMATRICULAUNIVERSIDAD")
Data['ESTU_VALORMATRICULAUNIVERSIDAD'] = Data['ESTU_VALORMATRICULAUNIVERSIDAD'].fillna('ValorNulo')
#Diccionario de mapeo para 'DESEMPEÑO_VLRMATRICULA'
desempeno_vlrmatricula_map = {
    'No pagó matrícula': 1,
    'Menos de 500 mil': 2,
    'Entre 500 mil y menos de 1 millón': 3,
    'Entre 1 millón y menos de 2.5 millones': 4,
    'Entre 2.5 millones y menos de 4 millones': 5,
    'Entre 4 millones y menos de 5.5 millones': 6,
    'Entre 5.5 millones y menos de 7 millones': 7,
    'Más de 7 millones': 8,
    'ValorNulo': 9
}
#Crear la nueva columna 'DESEMPEÑO_VLRMATRICULA' basada en 'ESTU_VALORMATRICULAUNIVERSIDAD'
Data['DESEMPEÑO_VLRMATRICULA'] = Data['ESTU_VALORMATRICULAUNIVERSIDAD'].map(desempeno_vlrmatricula_map)
#Verificar las primeras filas
print(Data[['ESTU_VALORMATRICULAUNIVERSIDAD', 'DESEMPEÑO_VLRMATRICULA']].head(10))
#Reemplazar los valores en 'ESTU_VALORMATRICULAUNIVERSIDAD' para simplificar
Data['ESTU_VALORMATRICULAUNIVERSIDAD'] = Data['ESTU_VALORMATRICULAUNIVERSIDAD'].replace({
    'No pagó matrícula': "Nivel0",
    'Menos de 500 mil': "Nivel1",
    'Entre 500 mil y menos de 1 millón': "Nivel2",
    'Entre 1 millón y menos de 2.5 millones': "Nivel3",
    'Entre 2.5 millones y menos de 4 millones': "Nivel4",
    'Entre 4 millones y menos de 5.5 millones': "Nivel5",
    'Entre 5.5 millones y menos de 7 millones': "Nivel6",
    'Más de 7 millones': "Nivel7",
    'ValorNulo': 'ValorNulo'
})
#Crear columnas binarias para cada nivel de 'ESTU_VALORMATRICULAUNIVERSIDAD' con prefijo 'VLRMATRICULA_'
Data = pd.get_dummies(Data, columns=['ESTU_VALORMATRICULAUNIVERSIDAD'], prefix='VLRMATRICULA')
#Filtrar las columnas que empiezan con 'VLRMATRICULA_'
vlrmatricula_columns = [col for col in Data.columns if col.startswith('VLRMATRICULA_')]
#Convertir valores True/False a 1 y 0 solo en las columnas filtradas
Data[vlrmatricula_columns] = Data[vlrmatricula_columns].astype(int)
#Verificar los cambios en las primeras 10 filas de cada nueva columna
for col in vlrmatricula_columns:
    print(f"{col}:\n", Data[col].head(10))
    print("\n" + "-"*40 + "\n")
print("--------------------------------------------------------")

#COLUMNA DESEMPEÑO_DPTO
print("COLUMNA DESEMPEÑO_DPTO")
#Diccionario de mapeo para 'DESEMPEÑO_DPTO'
desempeno_dpto_map = {
    'ATLANTICO': 1,
    'BOLIVAR': 2,
    'CESAR': 3,
    'CORDOBA': 4,
    'LA GUAJIRA': 5,
    'MAGDALENA': 6,
    'SUCRE': 7,
    'SAN ANDRES': 8,
    'ANTIOQUIA': 9,
    'BOYACA': 10,
    'CALDAS': 11,
    'CUNDINAMARCA': 12,
    'HUILA': 13,
    'NORTE SANTANDER': 14,
    'QUINDIO': 15,
    'RISARALDA': 16,
    'SANTANDER': 17,
    'TOLIMA': 18,
    'BOGOTÁ': 19,
    'CAUCA': 20,
    'CHOCO': 21,
    'NARIÑO': 22,
    'VALLE': 23,
    'ARAUCA': 24,
    'CASANARE': 25,
    'META': 26,
    'VICHADA': 27,
    'AMAZONAS': 28,
    'CAQUETA': 29,
    'GUAINIA': 30,
    'GUAVIARE': 31,
    'PUTUMAYO': 32,
    'VAUPES': 33
}
#Crear la nueva columna 'DESEMPEÑO_DPTO' basada en 'ESTU_PRGM_DEPARTAMENTO'
Data['DESEMPEÑO_DPTO'] = Data['ESTU_PRGM_DEPARTAMENTO'].map(desempeno_dpto_map)
#Verificar las primeras filas
print(Data[['ESTU_PRGM_DEPARTAMENTO', 'DESEMPEÑO_DPTO']].head(10))
#Crear columnas binarias para cada departamento en 'ESTU_PRGM_DEPARTAMENTO' con prefijo 'DPTO_'
Data = pd.get_dummies(Data, columns=['ESTU_PRGM_DEPARTAMENTO'], prefix='DPTO')
#Filtrar las columnas que empiezan con 'DPTO_'
dpto_columns = [col for col in Data.columns if col.startswith('DPTO_')]
#Convertir valores True/False a 1 y 0 solo en las columnas filtradas
Data[dpto_columns] = Data[dpto_columns].astype(int)
#Verificar los cambios en las primeras 10 filas de cada nueva columna
for col in dpto_columns:
    print(f"{col}:\n", Data[col].head(10))
    print("\n" + "-"*40 + "\n")
print("--------------------------------------------------------")

#COLUMNA ESTU_HORASSEMANATRABAJA
print("COLUMNA ESTU_HORASSEMANATRABAJA")
Data['ESTU_HORASSEMANATRABAJA'] = Data['ESTU_HORASSEMANATRABAJA'].fillna('ValorNulo')
#Diccionario de mapeo para 'DESEMPEÑO_HTRABAJO'
desempeno_htrabajo_map = {
    '0': 1,
    'Menos de 10 horas': 2,
    'Entre 11 y 20 horas': 3,
    'Entre 21 y 30 horas': 4,
    'Más de 30 horas': 5,
    'ValorNulo': 6
}
#Crear la nueva columna 'DESEMPEÑO_HTRABAJO' basada en 'ESTU_HORASSEMANATRABAJA'
Data['DESEMPEÑO_HTRABAJO'] = Data['ESTU_HORASSEMANATRABAJA'].map(desempeno_htrabajo_map)
#Verificar las primeras filas
print(Data[['ESTU_HORASSEMANATRABAJA', 'DESEMPEÑO_HTRABAJO']].head(10))
#Crear columnas binarias para cada rango de horas en 'ESTU_HORASSEMANATRABAJA' con prefijo 'HTRABAJO_'
Data = pd.get_dummies(Data, columns=['ESTU_HORASSEMANATRABAJA'], prefix='HTRABAJO')
#Filtrar las columnas que empiezan con 'HTRABAJO_'
htrabajo_columns = [col for col in Data.columns if col.startswith('HTRABAJO_')]
#Convertir valores True/False a 1 y 0 solo en las columnas filtradas
Data[htrabajo_columns] = Data[htrabajo_columns].astype(int)
#Verificar los cambios en las primeras 10 filas de cada nueva columna
for col in htrabajo_columns:
    print(f"{col}:\n", Data[col].head(10))
    print("\n" + "-"*40 + "\n")
print("--------------------------------------------------------")

#COLUMNA FAMI_ESTRATOVIVIENDA
print("COLUMNA FAMI_ESTRATOVIVIENDA")
Data['FAMI_ESTRATOVIVIENDA'] = Data['FAMI_ESTRATOVIVIENDA'].fillna('ValorNulo')
#Diccionario de mapeo para 'DESEMPEÑO_ESTRATO'
desempeno_estrato_map = {
    'Sin Estrato': 1,
    'Estrato 1': 2,
    'Estrato 2': 3,
    'Estrato 3': 4,
    'Estrato 4': 5,
    'Estrato 5': 6,
    'Estrato 6': 7,
    'ValorNulo': 8
}
#Crear la nueva columna 'DESEMPEÑO_ESTRATO' basada en 'FAMI_ESTRATOVIVIENDA'
Data['DESEMPEÑO_ESTRATO'] = Data['FAMI_ESTRATOVIVIENDA'].map(desempeno_estrato_map)
#Verificar las primeras filas
print(Data[['FAMI_ESTRATOVIVIENDA', 'DESEMPEÑO_ESTRATO']].head(10))
#Crear columnas binarias para cada nivel en 'FAMI_ESTRATOVIVIENDA' sin prefijo adicional
Data = pd.get_dummies(Data, columns=['FAMI_ESTRATOVIVIENDA'])
#Filtrar las columnas que comienzan con 'FAMI_ESTRATOVIVIENDA_'
estrato_columns = [col for col in Data.columns if col.startswith('FAMI_ESTRATOVIVIENDA_')]
#Convertir valores True/False a 1 y 0 solo en las columnas filtradas
Data[estrato_columns] = Data[estrato_columns].astype(int)
#Verificar los cambios en las primeras 10 filas de cada nueva columna
for col in estrato_columns:
    print(f"{col}:\n", Data[col].head(10))
    print("\n" + "-"*40 + "\n")
print("--------------------------------------------------------")

#COLUMNA FAMI_TIENEINTERNET
print("COLUMNA FAMI_TIENEINTERNET")
Data['FAMI_TIENEINTERNET'] = Data['FAMI_TIENEINTERNET'].fillna('ValorNulo')
#Diccionario de mapeo para 'DESEMPEÑO_INTERNET'
desempeno_internet_map = {
    'Si': 1,
    'No': 2,
    'ValorNulo': 3
}
#Crear la nueva columna 'DESEMPEÑO_INTERNET' basada en 'FAMI_TIENEINTERNET'
Data['DESEMPEÑO_INTERNET'] = Data['FAMI_TIENEINTERNET'].map(desempeno_internet_map)
#Verificar las primeras filas
print(Data[['FAMI_TIENEINTERNET', 'DESEMPEÑO_INTERNET']].head(10))
#Crear columnas binarias para cada valor en 'FAMI_TIENEINTERNET' con prefijo 'INTERNET'
Data = pd.get_dummies(Data, columns=['FAMI_TIENEINTERNET'], prefix='INTERNET')
#Filtrar las columnas que empiezan con 'INTERNET_'
internet_columns = [col for col in Data.columns if col.startswith('INTERNET_')]
#Convertir valores True/False a 1 y 0 solo en las columnas filtradas
Data[internet_columns] = Data[internet_columns].astype(int)
#Verificar los cambios en las primeras 10 filas de cada nueva columna
for col in internet_columns:
    print(f"{col}:\n", Data[col].head(10))
    print("\n" + "-"*40 + "\n")
print("--------------------------------------------------------")

#COLUMNAS AMI_EDUCACIONPADRE y FAMI_EDUCACIONMADRE
# Diccionario de reemplazos para las columnas FAMI_EDUCACIONPADRE y FAMI_EDUCACIONMADRE
reemplazos_educacion = {
    'Técnica o tecnológica incompleta': 'TTIncompleta',
    'Técnica o tecnológica completa': 'TTCompleta',
    'Secundaria (Bachillerato) completa': 'SecCompleta',
    'No sabe': 'NA',
    'Primaria completa': 'Pcompleta',
    'Educación profesional completa': 'ProfCompleta',
    'Educación profesional incompleta': 'ProfIncompleta',
    'Primaria incompleta': 'PIncompleta',
    'Postgrado': 'Postgrado',
    'Secundaria (Bachillerato) incompleta': 'SecIncompleta',
    'Ninguno': 'NA',
    'No Aplica': 'NA',
    np.nan: 'ValorNulo'
}
#Aplicar los reemplazos en ambas columnas
Data['FAMI_EDUCACIONPADRE'] = Data['FAMI_EDUCACIONPADRE'].replace(reemplazos_educacion)
Data['FAMI_EDUCACIONMADRE'] = Data['FAMI_EDUCACIONMADRE'].replace(reemplazos_educacion)
# Diccionario de mapeo para los valores educativos
educacion_map = {
    'Técnica o tecnológica incompleta': 1,
    'Técnica o tecnológica completa': 2,
    'Secundaria (Bachillerato) completa': 3,
    'No sabe': 4,
    'Primaria completa': 5,
    'Educación profesional completa': 6,
    'Educación profesional incompleta': 7,
    'Primaria incompleta': 8,
    'Postgrado': 9,
    'Secundaria (Bachillerato) incompleta': 10,
    'Ninguno': 11,
    'No Aplica': 12,
    'ValorNulo': 13
}
#Crear la nueva columna 'DESEMPEÑO_EPADRE' basada en 'FAMI_EDUCACIONPADRE'
Data['DESEMPEÑO_EPADRE'] = Data['FAMI_EDUCACIONPADRE'].map(educacion_map)
#Crear la nueva columna 'DESEMPEÑO_EMADRE' basada en 'FAMI_EDUCACIONMADRE'
Data['DESEMPEÑO_EMADRE'] = Data['FAMI_EDUCACIONMADRE'].map(educacion_map)
#Verificar las primeras filas de las columnas creadas
print(Data[['DESEMPEÑO_EPADRE', 'DESEMPEÑO_EMADRE']].head(10))
#Crear columnas binarias para 'FAMI_EDUCACIONPADRE' con prefijo 'EDUPADRE_'
Data = pd.get_dummies(Data, columns=['FAMI_EDUCACIONPADRE'], prefix='EDUPADRE')
#Crear columnas binarias para 'FAMI_EDUCACIONMADRE' con prefijo 'EDUMADRE_'
Data = pd.get_dummies(Data, columns=['FAMI_EDUCACIONMADRE'], prefix='EDUMADRE')
#Filtrar las columnas que empiezan con 'EDUPADRE_' y 'EDUMADRE_'
edupadre_columns = [col for col in Data.columns if col.startswith('EDUPADRE_')]
edumadre_columns = [col for col in Data.columns if col.startswith('EDUMADRE_')]
#Convertir valores True/False a 1 y 0 solo en las columnas filtradas
Data[edupadre_columns] = Data[edupadre_columns].astype(int)
Data[edumadre_columns] = Data[edumadre_columns].astype(int)
#Verificar los cambios en las primeras filas de cada nueva columna para 'FAMI_EDUCACIONPADRE'
print("Columnas binarias para FAMI_EDUCACIONPADRE:")
for col in edupadre_columns:
    print(f"{col}:\n", Data[col].head(10))
    print("\n" + "-"*40 + "\n")
#Verificar los cambios en las primeras filas de cada nueva columna para 'FAMI_EDUCACIONMADRE'
print("Columnas binarias para FAMI_EDUCACIONMADRE:")
for col in edumadre_columns:
    print(f"{col}:\n", Data[col].head(10))
    print("\n" + "-"*40 + "\n")
print("--------------------------------------------------------")

#COLUMNA ESTU_PAGOMATRICULAPROPIO
print("COLUMNA ESTU_PAGOMATRICULAPROPIO")
Data['ESTU_PAGOMATRICULAPROPIO'] = Data['ESTU_PAGOMATRICULAPROPIO'].fillna('ValorNulo')
#Diccionario de mapeo para 'DESEMPEÑO_PAGOM'
desempeno_pagom_map = {
    'Si': 1,
    'No': 2,
    'ValorNulo': 3
}
#Crear la nueva columna 'DESEMPEÑO_PAGOM' basada en 'ESTU_PAGOMATRICULAPROPIO'
Data['DESEMPEÑO_PAGOM'] = Data['ESTU_PAGOMATRICULAPROPIO'].map(desempeno_pagom_map)
#Verificar las primeras filas
print(Data[['ESTU_PAGOMATRICULAPROPIO', 'DESEMPEÑO_PAGOM']].head(10))
#Crear columnas binarias para cada valor en 'ESTU_PAGOMATRICULAPROPIO' con prefijo 'MLAPROPIO'
Data = pd.get_dummies(Data, columns=['ESTU_PAGOMATRICULAPROPIO'], prefix='MLAPROPIO')
#Filtrar las columnas que empiezan con 'MLAPROPIO_'
mlapropio_columns = [col for col in Data.columns if col.startswith('MLAPROPIO_')]
#Convertir valores True/False a 1 y 0 solo en las columnas filtradas
Data[mlapropio_columns] = Data[mlapropio_columns].astype(int)
#Verificar los cambios en las primeras 10 filas de cada nueva columna
for col in mlapropio_columns:
    print(f"{col}:\n", Data[col].head(10))
    print("\n" + "-"*40 + "\n")
print("--------------------------------------------------------")

#COLUMNA RENDIMIENTO_GLOBAL
print("COLUMNA RENDIMIENTO_GLOBAL")
#Crear un diccionario de reemplazo para los valores de RENDIMIENTO_GLOBAL
reemplazos_rendimiento = {
    'bajo': 1,
    'medio-bajo': 2,
    'medio-alto': 3,
    'alto': 4
}
#Reemplazar los valores en la columna RENDIMIENTO_GLOBAL
Data['RENDIMIENTO_GLOBAL'] = Data['RENDIMIENTO_GLOBAL'].replace(reemplazos_rendimiento)
#Verificar los primeros valores después del reemplazo
print(Data['RENDIMIENTO_GLOBAL'].head())
print("--------------------------------------------------------")




  Preparing metadata (setup.py) ... done
  Created wheel for rlxutils: filename=rlxutils-0.1.10-py3-none-any.whl size=11100 sha256=f93a5c7bc3cf718dbf9dc48f71c92b41c84b47d36f5fe6cd0f9a54cd2ad6c939
  Stored in directory: /root/.cache/pip/wheels/9a/45/da/49bdb0e82cc7a605e9c05dc24265687c5f349b53dcb74728e2
Successfully built rlxutils
 89% 18.0M/20.1M [00:01<00:00, 10.8MB/s]
100% 20.1M/20.1M [00:01<00:00, 10.7MB/s]
   PERIODO  DESEMPEÑO_PERIODO
0     2021                  4
1     2021                  4
2     2020                  3
3     2019                  2
4     2021                  4
5     2020                  3
6     2018                  1
7     2018                  1
8     2021                  4
9     2018                  1
PERIODO_2018:
 0    0
1    0
2    0
3    0
4    0
5    0
6    1
7    1
8    0
9    1
Name: PERIODO_2018, dtype: int64

----------------------------------------

PERIODO_2019:
 0    0
1    0
2    0
3    1
4    0
5    0
6    0
7    0
8    0
9    0
Name: PERIO

<ipython-input-1-a22b90da7845>:389: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  Data['RENDIMIENTO_GLOBAL'] = Data['RENDIMIENTO_GLOBAL'].replace(reemplazos_rendimiento)


In [ ]:
#Crear la columna PREDIC_DESEMP basada en los valores de RENDIMIENTO_GLOBAL
Data['PREDIC_DESEMP'] = Data['RENDIMIENTO_GLOBAL'].apply(lambda x: 0 if x in [1, 2] else 1)
#Verificar las primeras filas
print(Data[['RENDIMIENTO_GLOBAL', 'PREDIC_DESEMP']].head(10))

   RENDIMIENTO_GLOBAL  PREDIC_DESEMP
0                   3              1
1                   1              0
2                   1              0
3                   4              1
4                   2              0
5                   3              1
6                   4              1
7                   2              0
8                   2              0
9                   4              1


In [ ]:
!pip install catboost
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd

#Primer Modelo: En base a las columnas DESEMPEÑO_ y RENDIMIENTO GLOBAL, se propone entrenar un modelo
#Utilizando Catboost. En este caso se busca generar una columna denominada PREDIC_DESEMP, que asigna valores
#1 para Buen rendimiento (medio-alto y alto) y 0 para Bajo rendimiento (medio-bajo y bajo)

#Seleccionar columnas DESEMPEÑO_ como características
desempeno_columns = [col for col in Data.columns if col.startswith('DESEMPEÑO_')]
X_1 = Data[desempeno_columns]
y_1 = Data['PREDIC_DESEMP']

#Dividir datos en entrenamiento y prueba
X_train_1, X_test_1, y_train_1, y_test_1 = train_test_split(X_1, y_1, test_size=0.3, random_state=42)

#Entrenar el primer modelo
cat_model_1 = CatBoostClassifier(iterations=500, depth=6, learning_rate=0.1, random_seed=42, verbose=100)
cat_model_1.fit(X_train_1, y_train_1)

#Evaluar el primer modelo
y_pred_1 = cat_model_1.predict(X_test_1)
accuracy_1 = accuracy_score(y_test_1, y_pred_1)
report_1 = classification_report(y_test_1, y_pred_1)
print("Precisión del primer modelo CatBoost:", accuracy_1)
print("Reporte de clasificación para el primer modelo:\n", report_1)


#Segundo Modelo: Entrenar para predecir RENDIMIENTO_GLOBAL
#En este caso se tiene en cuenta la columna generada en el primer Modelo.
#Se descartan las columnas DESEMPEÑO


#Excluir columnas DESEMPEÑO_ y seleccionar columnas para el segundo modelo
columns_to_exclude = [col for col in Data.columns if col.startswith('DESEMPEÑO_')]
columns_to_use = [col for col in Data.columns if col not in columns_to_exclude + ['RENDIMIENTO_GLOBAL']]
columns_to_use = list(set(columns_to_use))  # Eliminar duplicados
if 'PREDIC_DESEMP' not in columns_to_use:
    columns_to_use.append('PREDIC_DESEMP')

#Crear el conjunto de características y la variable objetivo
X_2 = Data[columns_to_use]
y_2 = Data['RENDIMIENTO_GLOBAL']

#Dividir datos en entrenamiento y prueba
X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(X_2, y_2, test_size=0.3, random_state=42)

#Entrenar el segundo modelo
cat_model_2 = CatBoostClassifier(iterations=500, depth=6, learning_rate=0.1, random_seed=42, verbose=100)
cat_model_2.fit(X_train_2, y_train_2)

#Evaluar el segundo modelo
y_pred_2 = cat_model_2.predict(X_test_2)
accuracy_2 = accuracy_score(y_test_2, y_pred_2)
report_2 = classification_report(y_test_2, y_pred_2)
print("Precisión del segundo modelo CatBoost:", accuracy_2)
print("Reporte de clasificación para el segundo modelo:\n", report_2)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 6.0 MB/s eta 0:00:00
0:	learn: 0.6813529	total: 148ms	remaining: 1m 13s
100:	learn: 0.5960846	total: 12.2s	remaining: 48.4s
200:	learn: 0.5903068	total: 24.3s	remaining: 36.1s
300:	learn: 0.5876088	total: 36.8s	remaining: 24.3s
400:	learn: 0.5858079	total: 46.6s	remaining: 11.5s
499:	learn: 0.5844248	total: 59.1s	remaining: 0us
Precisión del primer modelo CatBoost: 0.6840625752105897
Reporte de clasificación para el primer modelo:
               precision    recall  f1-score   support

           0       0.67      0.73      0.70    103479
           1       0.70      0.64      0.67    104271

    accuracy                           0.68    207750
   macro avg       0.69      0.68      0.68    207750
weighted avg       0.69      0.68      0.68    207750

0:	learn: 1.2835119	total: 517ms	remaining: 4m 17s
100:	learn: 0.6467653	total: 27.2s	remaining: 1m 47s
200:	learn: 0.6423751	total: 54.4s	remaining: 1m 20s
300:	learn: 0.6400128	t

In [ ]:
#Cargar el archivo test.csv
zt = pd.read_csv("test.csv")
#Conservar los IDs para el archivo de envío
zt_ids = zt['ID'].values

#COLUMNA PERIODO
print("COLUMNA PERIODO")
#Modificar la columna 'PERIODO' en test.csv para que solo contenga el año
zt['PERIODO'] = zt['PERIODO'].astype(str).str[:4].astype(int)
zt['DESEMPEÑO_PERIODO'] = zt['PERIODO'].map(desempeno_periodo_map)
#Crear columnas binarias para cada año en la columna 'PERIODO' con prefijo 'PERIODO'
zt = pd.get_dummies(zt, columns=['PERIODO'], prefix='PERIODO')
#Filtrar las columnas que empiezan con 'PERIODO_'
periodo_columns_test = [col for col in zt.columns if col.startswith('PERIODO_')]
#Convertir valores True/False a 1 y 0 solo en las columnas filtradas
zt[periodo_columns_test] = zt[periodo_columns_test].astype(int)
#Verificar los cambios
for col in periodo_columns_test:
    print(f"{col} (test.csv):\n", zt[col].head(10))
    print("\n" + "-"*40 + "\n")
print("--------------------------------------------------------")

#COLUMNA ESTU_PRGM_ACADEMICO
print("COLUMNA ESTU_PRGM_ACADEMICO")
import unicodedata
import pandas as pd
#Función para eliminar tildes y normalizar el texto
def normalizar_texto(texto):
    if isinstance(texto, str):
        texto = texto.upper()
        texto = ''.join((c for c in unicodedata.normalize('NFD', texto) if unicodedata.category(c) != 'Mn'))
    return texto
#Cargar el archivo Excel "Programas.xlsx" y seleccionar la hoja "Sheet1"
programas_df = pd.read_excel('Programas.xlsx', sheet_name='Sheet1')
#Asegurarse de que los datos en la columna "ESTU_PRGM_ACADEMICO" estén normalizados
zt['ESTU_PRGM_ACADEMICO'] = zt['ESTU_PRGM_ACADEMICO'].apply(normalizar_texto)
#Normalizar también la columna 'Programa' y 'Campo de Conocimiento' en el archivo Excel
programas_df['Programa'] = programas_df['Programa'].apply(normalizar_texto)
programas_df['Campo de Conocimiento'] = programas_df['Campo de Conocimiento'].apply(normalizar_texto)
#Crear un diccionario para coincidencias exactas entre 'Programa' y 'Campo de Conocimiento'
reemplazos = dict(zip(programas_df['Programa'], programas_df['Campo de Conocimiento']))
#Reemplazar las coincidencias exactas en la columna 'ESTU_PRGM_ACADEMICO'
zt['ESTU_PRGM_ACADEMICO'] = zt['ESTU_PRGM_ACADEMICO'].replace(reemplazos)
#Verificar los primeros valores después del reemplazo
print(zt['ESTU_PRGM_ACADEMICO'].head(10))
#Imprimir todos los valores únicos de la columna ESTU_PRGM_ACADEMICO
valores_unicos_test = zt['ESTU_PRGM_ACADEMICO'].unique()
print("Valores únicos en ESTU_PRGM_ACADEMICO (test.csv):")
print(valores_unicos_test)
zt['DESEMPEÑO_PRGM_ACADEMICO'] = zt['ESTU_PRGM_ACADEMICO'].map(desempeno_prgm_map)
#Crear columnas binarias para cada programa en 'ESTU_PRGM_ACADEMICO' con prefijo 'PRGM_'
zt = pd.get_dummies(zt, columns=['ESTU_PRGM_ACADEMICO'], prefix='PRGM')
#Filtrar las columnas que empiezan con 'PRGM_'
program_columns_test = [col for col in zt.columns if col.startswith('PRGM_')]
#Convertir valores True/False a 1 y 0 solo en las columnas filtradas
zt[program_columns_test] = zt[program_columns_test].astype(int)
#verificar los cambios en las primeras 10 filas de cada nueva columna
for col in program_columns_test:
    print(f"{col} (test.csv):\n", zt[col].head(10))
    print("\n" + "-"*40 + "\n")
print("--------------------------------------------------------")

#COLUMNA ESTU_VALORMATRICULAUNIVERSIDAD
print("COLUMNA ESTU_VALORMATRICULAUNIVERSIDAD")
zt['ESTU_VALORMATRICULAUNIVERSIDAD'] = zt['ESTU_VALORMATRICULAUNIVERSIDAD'].fillna('ValorNulo')
zt['DESEMPEÑO_VLRMATRICULA'] = zt['ESTU_VALORMATRICULAUNIVERSIDAD'].map(desempeno_vlrmatricula_map)
#Reemplazar los valores en 'ESTU_VALORMATRICULAUNIVERSIDAD' para simplificar en test.csv
zt['ESTU_VALORMATRICULAUNIVERSIDAD'] = zt['ESTU_VALORMATRICULAUNIVERSIDAD'].replace({
    'No pagó matrícula': "Nivel0",
    'Menos de 500 mil': "Nivel1",
    'Entre 500 mil y menos de 1 millón': "Nivel2",
    'Entre 1 millón y menos de 2.5 millones': "Nivel3",
    'Entre 2.5 millones y menos de 4 millones': "Nivel4",
    'Entre 4 millones y menos de 5.5 millones': "Nivel5",
    'Entre 5.5 millones y menos de 7 millones': "Nivel6",
    'Más de 7 millones': "Nivel7",
    'ValorNulo': 'ValorNulo'
})
#Crear columnas binarias para cada nivel de 'ESTU_VALORMATRICULAUNIVERSIDAD' con prefijo 'VLRMATRICULA_'
zt = pd.get_dummies(zt, columns=['ESTU_VALORMATRICULAUNIVERSIDAD'], prefix='VLRMATRICULA')
#Filtrar las columnas que empiezan con 'VLRMATRICULA_'
vlrmatricula_columns_test = [col for col in zt.columns if col.startswith('VLRMATRICULA_')]
#Convertir valores True/False a 1 y 0 solo en las columnas filtradas
zt[vlrmatricula_columns_test] = zt[vlrmatricula_columns_test].astype(int)
#Verificar los cambios en las primeras 10 filas de cada nueva columna
for col in vlrmatricula_columns_test:
    print(f"{col} (test.csv):\n", zt[col].head(10))
    print("\n" + "-"*40 + "\n")
print("--------------------------------------------------------")

#COLUMNA DESEMPEÑO_DPTO
print("COLUMNA DESEMPEÑO_DPTO")
zt['DESEMPEÑO_DPTO'] = zt['ESTU_PRGM_DEPARTAMENTO'].map(desempeno_dpto_map)
#Crear columnas binarias para cada departamento en 'ESTU_PRGM_DEPARTAMENTO' con prefijo 'DPTO_'
zt = pd.get_dummies(zt, columns=['ESTU_PRGM_DEPARTAMENTO'], prefix='DPTO')
#Filtrar las columnas que empiezan con 'DPTO_'
dpto_columns_test = [col for col in zt.columns if col.startswith('DPTO_')]
#Convertir valores True/False a 1 y 0 solo en las columnas filtradas
zt[dpto_columns_test] = zt[dpto_columns_test].astype(int)
#Verificar los cambios en las primeras 10 filas de cada nueva columna
for col in dpto_columns_test:
    print(f"{col} (test.csv):\n", zt[col].head(10))
    print("\n" + "-"*40 + "\n")
print("--------------------------------------------------------")

#COLUMNA ESTU_HORASSEMANATRABAJA
print("COLUMNA ESTU_HORASSEMANATRABAJA")
zt['ESTU_HORASSEMANATRABAJA'] = zt['ESTU_HORASSEMANATRABAJA'].fillna('ValorNulo')
zt['DESEMPEÑO_HTRABAJO'] = zt['ESTU_HORASSEMANATRABAJA'].map(desempeno_htrabajo_map)
#Crear columnas binarias para cada rango de horas en 'ESTU_HORASSEMANATRABAJA' con prefijo 'HTRABAJO_'
zt = pd.get_dummies(zt, columns=['ESTU_HORASSEMANATRABAJA'], prefix='HTRABAJO')
#Filtrar las columnas que empiezan con 'HTRABAJO_'
htrabajo_columns_test = [col for col in zt.columns if col.startswith('HTRABAJO_')]
#Convertir valores True/False a 1 y 0 solo en las columnas filtradas
zt[htrabajo_columns_test] = zt[htrabajo_columns_test].astype(int)
#Verificar los cambios en las primeras 10 filas de cada nueva columna
for col in htrabajo_columns_test:
    print(f"{col} (test.csv):\n", zt[col].head(10))
    print("\n" + "-"*40 + "\n")
print("--------------------------------------------------------")

#COLUMNA FAMI_ESTRATOVIVIENDA
print("COLUMNA FAMI_ESTRATOVIVIENDA")
zt['FAMI_ESTRATOVIVIENDA'] = zt['FAMI_ESTRATOVIVIENDA'].fillna('ValorNulo')
zt['DESEMPEÑO_ESTRATO'] = zt['FAMI_ESTRATOVIVIENDA'].map(desempeno_estrato_map)
#Crear columnas binarias para cada nivel en 'FAMI_ESTRATOVIVIENDA' sin prefijo adicional
zt = pd.get_dummies(zt, columns=['FAMI_ESTRATOVIVIENDA'])
#Filtrar las columnas que comienzan con 'FAMI_ESTRATOVIVIENDA_'
estrato_columns_test = [col for col in zt.columns if col.startswith('FAMI_ESTRATOVIVIENDA_')]
#Convertir valores True/False a 1 y 0 solo en las columnas filtradas
zt[estrato_columns_test] = zt[estrato_columns_test].astype(int)
#Verificar los cambios en las primeras 10 filas de cada nueva columna
for col in estrato_columns_test:
    print(f"{col} (test.csv):\n", zt[col].head(10))
    print("\n" + "-"*40 + "\n")
print("--------------------------------------------------------")

#COLUMNA FAMI_TIENEINTERNET
print("COLUMNA FAMI_TIENEINTERNET")
zt['FAMI_TIENEINTERNET'] = zt['FAMI_TIENEINTERNET'].fillna('ValorNulo')
zt['DESEMPEÑO_INTERNET'] = zt['FAMI_TIENEINTERNET'].map(desempeno_internet_map)
#Crear columnas binarias para cada valor en 'FAMI_TIENEINTERNET' con prefijo 'INTERNET'
zt = pd.get_dummies(zt, columns=['FAMI_TIENEINTERNET'], prefix='INTERNET')
#Filtrar las columnas que empiezan con 'INTERNET_'
internet_columns_test = [col for col in zt.columns if col.startswith('INTERNET_')]
#Convertir valores True/False a 1 y 0 solo en las columnas filtradas
zt[internet_columns_test] = zt[internet_columns_test].astype(int)
#Verificar los cambios en las primeras 10 filas de cada nueva columna
for col in internet_columns_test:
    print(f"{col} (test.csv):\n", zt[col].head(10))
    print("\n" + "-"*40 + "\n")
print("--------------------------------------------------------")

#columnas FAMI_EDUCACIONPADRE y FAMI_EDUCACIONMADRE
print("columnas FAMI_EDUCACIONPADRE y FAMI_EDUCACIONMADRE")
#Reemplazar valores en las columnas FAMI_EDUCACIONPADRE y FAMI_EDUCACIONMADRE para zt
zt['FAMI_EDUCACIONPADRE'] = zt['FAMI_EDUCACIONPADRE'].replace(reemplazos_educacion)
zt['FAMI_EDUCACIONMADRE'] = zt['FAMI_EDUCACIONMADRE'].replace(reemplazos_educacion)

# Crear la nueva columna 'DESEMPEÑO_EPADRE' basada en 'FAMI_EDUCACIONPADRE'
zt['DESEMPEÑO_EPADRE'] = zt['FAMI_EDUCACIONPADRE'].map(educacion_map)
# Crear la nueva columna 'DESEMPEÑO_EMADRE' basada en 'FAMI_EDUCACIONMADRE'
zt['DESEMPEÑO_EMADRE'] = zt['FAMI_EDUCACIONMADRE'].map(educacion_map)
#Crear columnas binarias para 'FAMI_EDUCACIONPADRE' en zt con prefijo 'EDUPADRE_'
zt = pd.get_dummies(zt, columns=['FAMI_EDUCACIONPADRE'], prefix='EDUPADRE')
#Crear columnas binarias para 'FAMI_EDUCACIONMADRE' en zt con prefijo 'EDUMADRE_'
zt = pd.get_dummies(zt, columns=['FAMI_EDUCACIONMADRE'], prefix='EDUMADRE')
#Filtrar las columnas que empiezan con 'EDUPADRE_' y 'EDUMADRE_'
zt_edupadre_columns = [col for col in zt.columns if col.startswith('EDUPADRE_')]
zt_edumadre_columns = [col for col in zt.columns if col.startswith('EDUMADRE_')]
#Convertir valores True/False a 1 y 0 solo en las columnas filtradas
zt[zt_edupadre_columns] = zt[zt_edupadre_columns].astype(int)
zt[zt_edumadre_columns] = zt[zt_edumadre_columns].astype(int)
#Verificar los cambios en las primeras filas de cada nueva columna para 'FAMI_EDUCACIONPADRE' en zt
print("Columnas binarias para FAMI_EDUCACIONPADRE en zt:")
for col in zt_edupadre_columns:
    print(f"{col}:\n", zt[col].head(10))
    print("\n" + "-"*40 + "\n")
#Verificar los cambios en las primeras filas de cada nueva columna para 'FAMI_EDUCACIONMADRE' en zt
print("Columnas binarias para FAMI_EDUCACIONMADRE en zt:")
for col in zt_edumadre_columns:
    print(f"{col}:\n", zt[col].head(10))
    print("\n" + "-"*40 + "\n")
print("--------------------------------------------------------")

#COLUMNA ESTU_PAGOMATRICULAPROPIO
print("COLUMNA ESTU_PAGOMATRICULAPROPIO")
zt['ESTU_PAGOMATRICULAPROPIO'] = zt['ESTU_PAGOMATRICULAPROPIO'].fillna('ValorNulo')
zt['DESEMPEÑO_PAGOM'] = zt['ESTU_PAGOMATRICULAPROPIO'].map(desempeno_pagom_map)
#Crear columnas binarias para cada valor en 'ESTU_PAGOMATRICULAPROPIO' con prefijo 'MLAPROPIO'
zt = pd.get_dummies(zt, columns=['ESTU_PAGOMATRICULAPROPIO'], prefix='MLAPROPIO')
#Filtrar las columnas que empiezan con 'MLAPROPIO_'
mlapropio_columns_test = [col for col in zt.columns if col.startswith('MLAPROPIO_')]
#Convertir valores True/False a 1 y 0 solo en las columnas filtradas
zt[mlapropio_columns_test] = zt[mlapropio_columns_test].astype(int)
#Verificar los cambios en las primeras 10 filas de cada nueva columna
for col in mlapropio_columns_test:
    print(f"{col} (test.csv):\n", zt[col].head(10))
    print("\n" + "-"*40 + "\n")
print("--------------------------------------------------------")



COLUMNA PERIODO
PERIODO_2018 (test.csv):
 0    1
1    0
2    0
3    0
4    0
5    1
6    0
7    1
8    0
9    1
Name: PERIODO_2018, dtype: int64

----------------------------------------

PERIODO_2019 (test.csv):
 0    0
1    0
2    0
3    1
4    0
5    0
6    1
7    0
8    1
9    0
Name: PERIODO_2019, dtype: int64

----------------------------------------

PERIODO_2020 (test.csv):
 0    0
1    1
2    0
3    0
4    0
5    0
6    0
7    0
8    0
9    0
Name: PERIODO_2020, dtype: int64

----------------------------------------

PERIODO_2021 (test.csv):
 0    0
1    0
2    1
3    0
4    1
5    0
6    0
7    0
8    0
9    0
Name: PERIODO_2021, dtype: int64

----------------------------------------

--------------------------------------------------------
COLUMNA ESTU_PRGM_ACADEMICO
0          CIENCIAS SOCIALES Y HUMANIDADES
1    CIENCIAS ECONOMICAS Y ADMINISTRATIVAS
2          CIENCIAS NATURALES E INGENIERIA
3    CIENCIAS ECONOMICAS Y ADMINISTRATIVAS
4    CIENCIAS ECONOMICAS Y ADMINISTRATI

In [ ]:
#MODELO PARA TEST

#Asegurarse de que las columnas DESEMPEÑO_ existan en zt
for col in desempeno_columns:
    if col not in zt.columns:
        zt[col] = np.nan  # Crear columna si no existe
    zt[col].fillna(zt[col].median(), inplace=True)

#Predecir PREDIC_DESEMP con el primer modelo
zt['PREDIC_DESEMP'] = cat_model_1.predict(zt[desempeno_columns])

#Preprocesar las columnas necesarias para el segundo modelo
for col in columns_to_use:
    if col not in zt.columns:
        zt[col] = np.nan  # Crear columna si no existe
    if zt[col].dtype in ['float64', 'int64']:
        zt[col].fillna(zt[col].median(), inplace=True)
    else:
        zt[col].fillna('ValorNulo', inplace=True)

#Crear interacciones adicionales en zt
for col in desempeno_columns:
    zt[f'{col}_PREDIC_DESEMP'] = zt[col] * zt['PREDIC_DESEMP']

#Predecir RENDIMIENTO_GLOBAL con el segundo modelo
zt['RENDIMIENTO_GLOBAL_PRED'] = cat_model_2.predict(zt[columns_to_use])


#SE CREA EL ARCHIVO PARA ENVIAR A KAGLE

#Diccionario inverso para convertir las predicciones
reemplazos_rendimiento_inverso = {
    1: 'bajo',
    2: 'medio-bajo',
    3: 'medio-alto',
    4: 'alto'
}
zt['RENDIMIENTO_GLOBAL_TEXT'] = zt['RENDIMIENTO_GLOBAL_PRED'].map(reemplazos_rendimiento_inverso)

#Crear el DataFrame de envío
submission = zt[['ID', 'RENDIMIENTO_GLOBAL_TEXT']].copy()
submission.rename(columns={'RENDIMIENTO_GLOBAL_TEXT': 'RENDIMIENTO_GLOBAL'}, inplace=True)

#Guardar el archivo de envío
submission.to_csv('my_submission.csv', index=False)
print("Archivo 'my_submission.csv' creado exitosamente.")
print(submission.head())

<ipython-input-5-c8bf22ee4f39>:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  zt[col].fillna(zt[col].median(), inplace=True)
<ipython-input-5-c8bf22ee4f39>:19: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using '

Archivo 'my_submission.csv' creado exitosamente.


In [ ]:
!kaggle competitions submit -c udea-ai4eng-20242 -f my_submission.csv -m "Predicción con modelo CatBoost"

100% 4.14M/4.14M [00:02<00:00, 1.63MB/s]
Successfully submitted to UDEA/ai4eng 20242 - Pruebas Saber Pro Colombia